In [1]:
import torch 
import torch.nn as nn
import numpy as np
torch.manual_seed(0)
np.random.seed(0)
from predictor import ConvLayer, ReflectionPadConv, ResidualBlock


In [16]:
input = torch.randn((64,3,32,32))
c1 = ConvLayer(3,3,5,2)
c2 = ReflectionPadConv(3,3,5,2)
c1 = c1.eval()
c2 = c2.eval()
output1 = c1(input)
output2 = c2(input)
print(output1.shape)
print(output2.shape)
torch.testing.assert_close(output1, output2, rtol=1e-5, atol=1e-5)

torch.Size([64, 3, 16, 16])
torch.Size([64, 3, 16, 16])


AssertionError: Tensor-likes are not close!

Mismatched elements: 49152 / 49152 (100.0%)
Greatest absolute difference: 0.05875449627637863 at index (0, 2, 0, 0) (up to 1e-05 allowed)
Greatest relative difference: 4.754302501678467 at index (0, 2, 0, 0) (up to 1e-05 allowed)

In [17]:
print(torch.sum(torch.abs(output1-output2)))

tensor(2608.5713, grad_fn=<SumBackward0>)


In [12]:
from torch.fx import symbolic_trace

symbolic_traced = symbolic_trace(c1)

print(symbolic_traced.code)




def forward(self, x):
    reflection_pad = self.reflection_pad(x);  x = None
    conv2d = self.conv2d(reflection_pad);  reflection_pad = None
    return conv2d
    


In [2]:
from torch.fx import symbolic_trace
res_block = ResidualBlock(3)
print(symbolic_trace(res_block))

ResidualBlock(
  (in1): InstanceNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
  (relu): LeakyReLU(negative_slope=0.1, inplace=True)
  (conv1): Module(
    (padding): ReflectionPad2d((1, 1, 1, 1))
    (conv): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  )
  (in2): InstanceNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
  (conv2): Module(
    (padding): ReflectionPad2d((1, 1, 1, 1))
    (conv): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  )
)



def forward(self, x):
    in1 = self.in1(x)
    relu = self.relu(in1);  in1 = None
    conv1_padding = self.conv1.padding(relu);  relu = None
    conv1_conv = self.conv1.conv(conv1_padding);  conv1_padding = None
    in2 = self.in2(conv1_conv);  conv1_conv = None
    relu_1 = self.relu(in2);  in2 = None
    conv2_padding = self.conv2.padding(relu_1);  relu_1 = None
    conv2_conv = self.conv2.conv(conv2_padding);  conv2_padding = None
    add = conv2_conv + x;  conv2_conv = x = Non

In [13]:
tp = 4409
fp = 5627
fn = 6841
tn = 26323
precision_0 = tn / (tn+fn)
recall_0 = tn / (tn+fp)
precision_1 = tp / (tp+fp)
recall_1 = tp / (tp+fn)
f1_0 = 2*(recall_0*precision_0)/(recall_0+precision_0)
f1_1 = 2*(recall_1*precision_1)/(recall_1+precision_1)

print('precision',precision_0, precision_1)
print('recall',recall_0, recall_1)
print('f1',f1_0, f1_1)


precision 0.793722108310216 0.4393184535671582
recall 0.8238810641627543 0.3919111111111111
f1 0.8085204410725804 0.4142628958000564


In [26]:
x = torch.randn((1,2,3,4))
z = torch.randn((2,2))
z = z.view(*z.shape,1,1).expand(*z.shape,x.shape[2],x.shape[3])
# z = z.view(z.size(0), z.size(1), 1, 1).expand(z.shape[0], z.shape[1], x.shape[0], x.shape[1])
print(z.shape)

torch.Size([2, 2, 3, 4])


In [8]:
def gram_matrix(input_tensor):
    # Get the dimensions of the input tensor
    batch_size, num_features, height, width = input_tensor.size()

    # Reshape the input tensor to a 2D matrix of shape (num_features, height * width)
    features = input_tensor.view(batch_size * num_features, height * width)

    # Calculate the Gram matrix by multiplying the reshaped features matrix with its transpose
    gram = torch.matmul(features, features.t())

    # Normalize the Gram matrix by dividing by the number of elements in each feature map
    gram /= (batch_size * num_features * height * width)

    return gram
def gram2(input):
    a, b, c, d = input.size()  # a=batch size(=1)
    # b=number of feature maps
    # (c,d)=dimensions of a f. map (N=c*d)

    features = input.view(a * b, c * d)  # resise F_XL into \hat F_XL

    G = torch.mm(features, features.t())  # compute the gram product
    # we 'normalize' the values of the gram matrix
    # by dividing by the number of element in each feature maps.
    return G.div(a * b * c * d)

In [9]:
import torch
features = torch.randn((1,2,3,4))
f1 = gram_matrix(features)
f2 = gram2(features)
torch.testing.assert_close(f1, f2)

In [5]:
from torch.fx import symbolic_trace
from allsome import *
nl_layer = get_non_linearity(layer_type='lrelu')
norm_layer = get_norm_layer(layer_type='batch')
netE = E_ResNet(input_nc=3, output_nc=1, ndf=64, n_blocks=4, norm_layer=norm_layer,
                        nl_layer=nl_layer, gpu_ids=[], vaeLike=False)
sample_input = torch.randn(64, 3, 128, 128)
output = netE(sample_input)
print(output.shape)

torch.Size([64, 1])


In [12]:
import glob
dir = '/Users/ganeshanmalhotra/Desktop/Quarter3/Visual/VisualLearningProject/sky_timelapse/sky_test'
dir_list = glob.glob(dir + '/*' + '/*')
for dir in dir_list:
    fnames = glob.glob(dir+"/*.jpg")
    print(dir, len(fnames))

/Users/ganeshanmalhotra/Desktop/Quarter3/Visual/VisualLearningProject/sky_timelapse/sky_test/i1nIvaejHZg/i1nIvaejHZg_1 2311
/Users/ganeshanmalhotra/Desktop/Quarter3/Visual/VisualLearningProject/sky_timelapse/sky_test/AF9A0Raa63M/AF9A0Raa63M_1 91
/Users/ganeshanmalhotra/Desktop/Quarter3/Visual/VisualLearningProject/sky_timelapse/sky_test/AF9A0Raa63M/AF9A0Raa63M_2 76
/Users/ganeshanmalhotra/Desktop/Quarter3/Visual/VisualLearningProject/sky_timelapse/sky_test/av0WFwFMB8g/av0WFwFMB8g_1 796
/Users/ganeshanmalhotra/Desktop/Quarter3/Visual/VisualLearningProject/sky_timelapse/sky_test/0_eqVxGOjuI/0_eqVxGOjuI_1 511
/Users/ganeshanmalhotra/Desktop/Quarter3/Visual/VisualLearningProject/sky_timelapse/sky_test/2jI_2QvtEQE/2jI_2QvtEQE_1 222
/Users/ganeshanmalhotra/Desktop/Quarter3/Visual/VisualLearningProject/sky_timelapse/sky_test/3aegvVRcBqI/3aegvVRcBqI_1 177
/Users/ganeshanmalhotra/Desktop/Quarter3/Visual/VisualLearningProject/sky_timelapse/sky_test/fsa7x8JsFx4/fsa7x8JsFx4_1 961
/Users/ganeshanma

In [8]:
len(fnames)

571

In [15]:
model = nn.Sequential( nn.LeakyReLU(0.2, inplace=True), 
                       nn.Conv2d(64,128, kernel_size=3, stride=1, padding=1),
                       nn.LeakyReLU(0.2, inplace=True),
                       nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
                       nn.AvgPool2d(kernel_size=2, stride=2, padding=0))

x = torch.randn(1, 64, 64, 64)
y = model(x)
print(y.shape)

RuntimeError: Given groups=1, weight of size [128, 64, 3, 3], expected input[1, 128, 64, 64] to have 64 channels, but got 128 channels instead